In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import hackathon_linc as lh
import kymatio as kt
import torch as th
import scipy as sp
print("import success")

import success


In [2]:
def load_csv_to_df(csv_path):
    """
    Load the CSV file into a pandas DataFrame.
    
    Parameters:
        csv_path (str): The path to the CSV file.
        
    Returns:
        pd.DataFrame: DataFrame containing the CSV data.
    """
    df = pd.read_csv(csv_path)
    return df


if __name__ == "__main__":
    df = load_csv_to_df('stockPrices_hourly.csv')

df    

,gmtTime,askMedian,bidMedian,askVolume,bidVolume,spreadMedian,symbol
0,2015-04-22 07:00:00,31.563367,31.549133,3.401711e+05,2.382823e+05,0.014100,STOCK5
1,2015-04-22 07:00:00,92.565750,92.520333,7.772792e+04,6.674355e+04,0.045450,STOCK2
2,2015-04-22 07:00:00,112.642667,112.560167,1.248314e+05,1.402586e+05,0.082542,STOCK3
3,2015-04-22 07:00:00,116.097050,115.896233,5.192117e+03,7.605817e+03,0.200833,STOCK4
4,2015-04-22 07:00:00,11.846933,11.836850,5.298832e+05,3.301907e+05,0.010050,STOCK7
...,...,...,...,...,...,...,...
118285,2020-04-23 15:00:00,108.880000,108.824667,6.949163e+04,5.707417e+06,0.055300,STOCK8
118286,2020-04-23 15:00:00,50.393900,50.363700,1.963220e+06,3.780463e+05,0.030117,STOCK3
118287,2020-04-23 15:00:00,24.438600,24.400200,4.022667e+03,6.601000e+05,0.038383,STOCK4
118288,2020-04-23 15:00:00,5.845367,5.837467,1.047933e+03,1.320250e+06,0.007800,STOCK5


#### Below is a test to see if the csv is sorted and valid or not.

It seems valid!

In [7]:
import pandas as pd
import numpy as np

# Load the CSV file with parsed datetime column 'gmtTime'
df = load_csv_to_df('stockPrices_hourly.csv')

# Display basic info
print("CSV Columns:", df.columns.tolist())
print("Number of rows:", len(df))
df.head()

# Checking Data Sorting by Time
is_sorted = df['gmtTime'].is_monotonic_increasing
print("Data sorted by gmtTime:", is_sorted)

# If not, sort the data
if not is_sorted:
    df = df.sort_values('gmtTime')
    print("Data has been re-sorted by gmtTime.")

# Checking the total timespan
min_time = df['gmtTime'].min()
max_time = df['gmtTime'].max()
print(f"Data spans from {min_time} to {max_time}.")

# Check the unique tickers in the CSV
ticker_col = df.columns[-1]  # Assuming the last column is the ticker identifier
tickers = df[ticker_col].unique()
print("Unique tickers:", tickers)
print("Number of tickers:", len(tickers))

# Number of elements over time
time_counts = df['gmtTime'].value_counts().sort_index()
print("Number of time points recorded:", len(time_counts))

# Display count statistics
print("Min elements per time point:", time_counts.min())
print("Max elements per time point:", time_counts.max())
print("Mean elements per time point:", time_counts.mean())

# Group the data by ticker and show the first 5 rows for each ticker
grouped = df.groupby(ticker_col)
for ticker, group in grouped:
    print(f"\nTicker: {ticker}")
    display(group.head())

# Compute the time range for each ticker
ticker_time_ranges = {}
for ticker, group in grouped:
    min_time = group['gmtTime'].min()
    max_time = group['gmtTime'].max()
    num_entries = len(group)
    ticker_time_ranges[ticker] = (min_time, max_time, num_entries)
    print(f"Ticker {ticker}: {num_entries} data points from {min_time} to {max_time}")
    
ticker_time_ranges_df = pd.DataFrame(ticker_time_ranges, index=['Start', 'End', 'Count']).T

display(ticker_time_ranges_df)

# Pivot the DataFrame so each ticker's 'askMedian' is in a separate column
pivot_df = df.pivot(index='gmtTime', columns=ticker_col, values='askMedian')

# Check the global time index of the pivoted DataFrame
print("Pivot DataFrame time index is sorted:", pivot_df.index.is_monotonic_increasing)
display(pivot_df.head())

CSV Columns: ['gmtTime', 'askMedian', 'bidMedian', 'askVolume', 'bidVolume', 'spreadMedian', 'symbol']
Number of rows: 118290
Data sorted by gmtTime: True
Data spans from 2015-04-22 07:00:00 to 2020-04-23 15:00:00.
Unique tickers: ['STOCK5' 'STOCK2' 'STOCK3' 'STOCK4' 'STOCK7' 'STOCK6' 'STOCK8' 'STOCK9'
 'STOCK10' 'STOCK1']
Number of tickers: 10
Number of time points recorded: 11878
Min elements per time point: 1
Max elements per time point: 10
Mean elements per time point: 9.958747263849133

Ticker: STOCK1


,gmtTime,askMedian,bidMedian,askVolume,bidVolume,spreadMedian,symbol
18,2015-04-22 08:00:00,75.060467,75.027900,658.950000,418.816667,0.032500,STOCK1
19,2015-04-22 09:00:00,74.813583,74.784950,390.616667,321.366667,0.028667,STOCK1
30,2015-04-22 10:00:00,74.508883,74.477000,498.749997,317.700000,0.031825,STOCK1
42,2015-04-22 11:00:00,74.515350,74.486233,250.116667,286.550000,0.029075,STOCK1
52,2015-04-22 12:00:00,74.435867,74.410100,357.583333,373.900000,0.025808,STOCK1



Ticker: STOCK10


,gmtTime,askMedian,bidMedian,askVolume,bidVolume,spreadMedian,symbol
8,2015-04-22 07:00:00,229.335983,229.185733,49437.383292,42733.383450,0.150183,STOCK10
10,2015-04-22 08:00:00,227.053500,226.954200,94588.850367,72674.516430,0.099258,STOCK10
27,2015-04-22 09:00:00,226.618000,226.525700,86416.667078,59030.783672,0.092342,STOCK10
37,2015-04-22 10:00:00,226.346417,226.259167,77002.733352,50582.633198,0.087233,STOCK10
43,2015-04-22 11:00:00,227.215733,227.127450,62320.299937,51052.066195,0.088283,STOCK10



Ticker: STOCK2


...

,gmtTime,askMedian,bidMedian,askVolume,bidVolume,spreadMedian,symbol
7,2015-04-22 07:00:00,100.758050,100.674283,132138.334163,125736.682848,0.083767,STOCK9
16,2015-04-22 08:00:00,100.094333,100.032467,165504.883543,132927.267008,0.061775,STOCK9
21,2015-04-22 09:00:00,99.753000,99.720967,70320.967023,70241.833285,0.032008,STOCK9
36,2015-04-22 10:00:00,99.531233,99.499983,79385.816430,79798.100270,0.031308,STOCK9
47,2015-04-22 11:00:00,99.730550,99.701900,55014.733475,54251.966400,0.028625,STOCK9


Ticker STOCK1: 11857 data points from 2015-04-22 08:00:00 to 2020-04-23 15:00:00
Ticker STOCK10: 11863 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK2: 11840 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK3: 11876 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK4: 11788 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK5: 11861 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK6: 11856 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK7: 11735 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK8: 11871 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00
Ticker STOCK9: 11743 data points from 2015-04-22 07:00:00 to 2020-04-23 15:00:00


,Start,End,Count
STOCK1,2015-04-22 08:00:00,2020-04-23 15:00:00,11857
STOCK10,2015-04-22 07:00:00,2020-04-23 15:00:00,11863
STOCK2,2015-04-22 07:00:00,2020-04-23 15:00:00,11840
STOCK3,2015-04-22 07:00:00,2020-04-23 15:00:00,11876
STOCK4,2015-04-22 07:00:00,2020-04-23 15:00:00,11788
STOCK5,2015-04-22 07:00:00,2020-04-23 15:00:00,11861
STOCK6,2015-04-22 07:00:00,2020-04-23 15:00:00,11856
STOCK7,2015-04-22 07:00:00,2020-04-23 15:00:00,11735
STOCK8,2015-04-22 07:00:00,2020-04-23 15:00:00,11871
STOCK9,2015-04-22 07:00:00,2020-04-23 15:00:00,11743


Pivot DataFrame time index is sorted: True


symbol,STOCK1,STOCK10,STOCK2,STOCK3,STOCK4,STOCK5,STOCK6,STOCK7,STOCK8,STOCK9
gmtTime,,,,,,,,,,
2015-04-22 07:00:00,NaN,229.335983,92.565750,112.642667,116.097050,31.563367,111.434283,11.846933,69.51030,100.758050
2015-04-22 08:00:00,75.060467,227.053500,91.635517,111.459883,115.118667,31.456000,110.594533,11.734917,68.96225,100.094333
2015-04-22 09:00:00,74.813583,226.618000,91.221183,110.351050,114.705917,31.354000,109.814633,11.686850,68.60045,99.753000
2015-04-22 10:00:00,74.508883,226.346417,90.940483,109.370150,114.189233,31.213100,109.289217,11.661617,68.36120,99.531233
2015-04-22 11:00:00,74.515350,227.215733,91.331583,110.055483,114.243917,31.360233,109.457650,11.712750,68.58370,99.730550
